<a href="https://colab.research.google.com/github/muhammadanas25/Stock_trend_prediction/blob/main/Extra_Tree_Stock_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Initial imports
import pandas as pd
!pip install path
from path import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline
# Initial imports
import pandas as pd
import numpy as np
from path import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier
import plotly.graph_objects as go

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

from sklearn.compose import make_column_selector as selector


In [ ]:
# Loading data
file_path = Path("ORCL.csv")
df = pd.read_csv(file_path)
df.dropna(inplace=True)

df=df[df.FutureTrend !='Neutral']
df.tail()

,Date,Open,High,Low,Close,Volume,EF_1,EF_2,Oil FutureTrend,Gold FutureTrend,Trend,Twitter Sentiments,FutureTrend
749,23/06/2021,78.760002,78.870003,78.059998,78.099998,7295900,0.049384,-0.837993,Negative,Positive,Negative,0,Positive
750,24/06/2021,78.650002,78.959999,77.610001,77.739998,9624000,0.096294,-1.157029,Negative,Negative,Negative,0,Negative
751,25/06/2021,78.029999,78.610001,77.559998,78.459999,12845000,0.857142,0.551071,Positive,Negative,Positive,0,Negative
752,28/06/2021,78.489998,78.970001,78.330002,78.500000,5605400,0.265622,0.012743,Negative,Positive,Positive,0,Positive
753,29/06/2021,78.540001,78.589996,77.820000,78.099998,5392900,0.363636,-0.560227,Negative,Negative,Negative,0,Positive


In [ ]:
def make_bins(df):
   label_names = ["Ef_1_v_low","Ef_1_low","Ef_1_v_med","Ef1_high", "Ef1_very_high" ]
   cut_points =[0. , 0.2, 0.4, 0.6, 0.8, 1. ]
   df["EF_1_val"] = pd.cut(df["EF_1"], cut_points, labels=label_names)
   dummies = pd.get_dummies(df["EF_1_val"],drop_first=True)
   df = pd.concat([df, dummies], axis=1)
   label_names = ["setinent_neutral","sentiment_good","sentiment_bad"]
   cut_points =[0, 0.29,0.58 ,0.87] 
   df["sentiments"]=pd.cut(df['Twitter Sentiments'], cut_points, labels=label_names)
   dummies1 = pd.get_dummies(df["sentiments"],drop_first=True)
   df = pd.concat([df, dummies1], axis=1)
   df.drop(["EF_1_val","sentiments"],axis=1,inplace=True)
   return df
def calculate_ema(prices, days, smoothing=2):
      ema = [sum(prices[:days]) / days]
      for price in prices[days:]:
          ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
      return ema

In [ ]:

target=df['FutureTrend']
df=df.drop("FutureTrend",axis =1)
target= target.replace(['Negative','Positive',  'Neutral'],[0,1,2])
#df= df.replace(['Negative','Positive',  'Neutral'],[0,1,2])
appl_df =df.set_index('Date',inplace=True)
  
appl_df = make_bins(df)
appl_df.head(2)
appl_df=appl_df.drop(["Twitter Sentiments"],axis =1)
appl_df['diff']=appl_df['Close']-appl_df['Open']
appl_df['10day']=appl_df['Close'].rolling(10).mean()
appl_df['10day'].fillna(appl_df['10day'][9],inplace=True)
  
ema = calculate_ema(appl_df['Close'], 10)
appl_df=appl_df[9:]
target=target[9:]
appl_df['ema']=ema
appl_df['signal']=appl_df['ema']>appl_df['Close']

del (appl_df['Open'])
del (appl_df['Close'])
del (appl_df['High'])
del (appl_df['EF_2'])


In [ ]:
new_df=appl_df
print(new_df.head())
new_df= new_df.replace(['Negative','Positive',  'Neutral'],[0,1,2])
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)
numerical_columns = numerical_columns_selector(new_df)
categorical_columns = categorical_columns_selector(new_df)
print(categorical_columns)
preprocessor = ColumnTransformer([
  # ('one-hot-encoder', categorical_preprocessor, categorical_columns),
   ('standard-scaler', numerical_preprocessor, numerical_columns)
    ])

model_2 = make_pipeline(preprocessor,ExtraTreesClassifier(n_estimators = 3,
                                        criterion ='gini', max_features = 2))

     

X_train,X_test, y_train, y_test = train_test_split(new_df, target, test_size=0.2, random_state=42)
model_2.fit(X_train,y_train)

  # Scoring the model_2
print("Accuracy score (training): {0:.3f}".format(
        model_2.score(
            X_train, 
            y_train)))
print("Accuracy score (validation): {0:.3f}".format(
        model_2.score(
            X_test, 
            y_test)))

                  Low    Volume      EF_1 Oil FutureTrend Gold FutureTrend  \
Date                                                                         
16/07/2018  47.950001  22949600  0.761194        Negative         Positive   
17/07/2018  48.150002  18823100  0.892857        Negative         Negative   
18/07/2018  48.400002  19234700  0.399997        Negative         Positive   
19/07/2018  48.209999  16656500  0.450978        Negative         Negative   
20/07/2018  48.360001  17232900  0.457144        Positive         Positive   

               Trend  Ef_1_low  Ef_1_v_med  Ef1_high  Ef1_very_high  \
Date                                                                  
16/07/2018  Negative         0           0         1              0   
17/07/2018  Positive         0           0         0              1   
18/07/2018  Negative         1           0         0              0   
19/07/2018  Positive         0           1         0              0   
20/07/2018  Positive       

In [ ]:
# |%pip install lazypredict
# #X = train.drop(['Class'], axis = 1)
# # y = train['Class']
# import lazypredict
# from lazypredict.Supervised import LazyClassifier
# clf = LazyClassifier(verbose=0,ignore_warnings=True)
# scores = []
# X_train,X_test, y_train, y_test = train_test_split(new_df, target,  test_size=.2, random_state =123)
# models,y_hat=clf.fit(X_train, X_test,y_train, y_test)
# print(models)